# Implementando um modelo de Recomendação Hibrido

Vamos construir um sistema de recomendação híbrido usando a implementação do python chamada LightFM. Nesta implementação, veremos como podemos estimar o modelo dado acima (BPR e WARP). Antes de ir para a implementação, podemos exigir a instalação da biblioteca que devemos usar na implementação

In [ ]:
!pip install lightfm -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.1/310.1 KB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import pandas as pd
import numpy as np
from lightfm.datasets import fetch_movielens

In [ ]:
data = fetch_movielens()

In [ ]:
#Checking the dictionaries and their size in  the data:
for key, value in data.items():
    print(key, value.shape)

train (943, 1682)
test (943, 1682)
item_features (1682, 1682)
item_feature_labels (1682,)
item_labels (1682,)


In [ ]:
#Defining the train and test data for training and testing purposes:
train = data['train']
test = data['test']

## Perdas usadas pelos Sistemas de Recomendação

###Bayesian Personalised Ranking(BPR) pairwise loss 

Este método pode ser usado quando a interação positiva do usuário nos dados é apresentada e somos obrigados a otimizar o ROC AUC. neste usando a perda pairwise, tentamos maximizar a diferença de previsão entre o feedback positivo e um feedback negativo selecionado aleatoriamente.

###Weighted Approximate-Rank Pairwise(WARP) loss
Isso é útil quando a interação positiva está disponível no feedback e somos obrigados a otimizar algumas das principais recomendações. Aqui, ele amostra repetidamente o feedback negativo até encontrar o feedback que está violando a classificação e esse procedimento maximiza a classificação do feedback positivo.

In [ ]:
from lightfm import LightFM
model = LightFM(learning_rate=0.05, loss='warp', learning_schedule='adagrad', no_components = 20)

In [ ]:
model.fit(train, epochs=10)

In [ ]:
from lightfm.evaluation import precision_at_k, auc_score

In [ ]:
train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()
train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test, train_interactions=train).mean()

In [ ]:
print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Precision: train 0.64, test 0.24.
AUC: train 0.95, test 0.93.


Maciej Kula - Hybrid Recommender System in Python